In [29]:
import cobra
import pandas as pd
import numpy as np
import os
import glob
from cobra import Model, Reaction, Metabolite
import subprocess
from cobra.flux_analysis.parsimonious import add_pfba
from cobra import test
print (cobra.__version__)

model = cobra.test.create_test_model("textbook")

0.13.4


In [38]:
model.genes

[<Gene b1241 at 0x11a811668>,
 <Gene b0351 at 0x11a8114a8>,
 <Gene s0001 at 0x11a811208>,
 <Gene b3115 at 0x11a8110b8>,
 <Gene b1849 at 0x11a78bf98>,
 <Gene b2296 at 0x11a78bb38>,
 <Gene b1276 at 0x11a78beb8>,
 <Gene b0118 at 0x11a78bc88>,
 <Gene b0474 at 0x11a78bcc0>,
 <Gene b0116 at 0x11a78bc18>,
 <Gene b0726 at 0x11a78b9b0>,
 <Gene b0727 at 0x11a78b5f8>,
 <Gene b2587 at 0x11a78b710>,
 <Gene b0356 at 0x11a78b6d8>,
 <Gene b1478 at 0x11a78b128>,
 <Gene b3735 at 0x11a78b4a8>,
 <Gene b3733 at 0x11a78b160>,
 <Gene b3734 at 0x11a78b1d0>,
 <Gene b3732 at 0x11a77c748>,
 <Gene b3736 at 0x11a77c860>,
 <Gene b3738 at 0x11a77cf28>,
 <Gene b3731 at 0x11a77cba8>,
 <Gene b3737 at 0x11a77cd68>,
 <Gene b3739 at 0x11a77cc88>,
 <Gene b0720 at 0x11a77c048>,
 <Gene b0733 at 0x11a77c9b0>,
 <Gene b0979 at 0x11a77c780>,
 <Gene b0978 at 0x11a77c898>,
 <Gene b0734 at 0x11a77c908>,
 <Gene b2975 at 0x11a77c1d0>,
 <Gene b3603 at 0x11a77c5f8>,
 <Gene b2779 at 0x11a77c390>,
 <Gene b1773 at 0x11a77c0b8>,
 <Gene b29

array([[0.05, 0.05],
       [0.05, 0.05],
       [0.05, 0.05],
       [0.05, 0.05],
       [0.05, 0.05],
       [0.05, 0.05],
       [0.5 , 0.5 ]])

In [42]:
### FROM DRIVEN driven/src/driven/data_sets/expression_profile.py

from __future__ import absolute_import

from itertools import combinations

import numpy as np
import pandas as pd
from sympy import Min, Max, Add, Mul, Symbol
from sympy.parsing.ast_parser import parse_expr

class ExpressionProfile(object):
    """
    Representation of an expression profile.
    It can be RNA-Seq, Proteomics, TNSeq or any other profile that
    links genes/proteins to a value (continuous or discrete). It is
    the storage of single or multiple conditions as well as p-values.
    Attributes
    ----------
    identifiers: list
        The gene/protein IDs.
    conditions: list
        The conditions in the expression profile (e.g., time points,
        media conditions, etc.)
    expression: numpy.ndarray
        A 2-dimensional array having no. of genes/proteins as rows and
        no. of conditions as the columns.
    p_values: numpy.ndarray
        The p-values between conditions.
    """

    def __init__(self, identifiers, conditions, expression, p_values=None):
        """
        Instantiate a new ExpressionProfile.
        Parameters
        ----------
        identifiers: list or array-like
            Genes/Proteins for the dataset.
        conditions: list or array-like
            Conditions for the dataset.
        expression: numpy.ndarray
            Expression data being used.
        p_values: numpy.ndarray, optional (default None)
            p-values obtained from the expresion data.
        """
        array_error = "Not an array-like structure."
        dimension_error = "Expression data and label dimensions don't match."
        assert isinstance(identifiers, (list, np.ndarray)), array_error
        assert isinstance(conditions, (list, np.ndarray)), array_error
        assert isinstance(expression, np.ndarray), array_error
        assert isinstance(p_values, (type(None), np.ndarray)), array_error
        dimension = (len(identifiers), len(conditions))
        if len(identifiers) == 1:
            assert expression.shape[0] == len(conditions)
        else:
            assert expression.shape == dimension, dimension_error

        self.identifiers = identifiers
        self.identifier_index = {iden: idx for idx, iden in
                                 enumerate(identifiers)}
        self.conditions = conditions
        self._condition_index = {cond: idx for idx, cond in
                                 enumerate(conditions)}
        self.expression = expression
        self._p_values = p_values

    def __getitem__(self, item):
        """
        Index the ExpressionProfile.
        Parameters
        ----------
        item: [str, str] or [int, int] or [slice, slice]
        """
        attribute_error = ("Non-supported indexing method. "
                           "Use profile['gene', 'condition'] "
                           "or profile[1, 2].")
        if not isinstance(item, tuple):
            raise AttributeError(attribute_error)

        if isinstance(item[0], str) and isinstance(item[1], str):
            i = self.identifier_index[item[0]]
            j = self._condition_index[item[1]]
        elif isinstance(item[0], (slice, int)) and isinstance(item[1],
                                                              (slice, int)):
            i = item[0]
            j = item[1]
        else:
            raise AttributeError(attribute_error)

        return self.expression[i, j]

    def __eq__(self, other):
        """
        Check equality of two ExpressionProfiles.
        Parameters
        ----------
        other: ExpressionProfile or other object
            The object to check equality against.
        """
        if not isinstance(other, ExpressionProfile):
            return False
        else:
            if self._p_values is None and other.p_values is None:
                return self.identifiers == other.identifiers and \
                       self.conditions == other.conditions and \
                       self._p_values == other._p_values and \
                       (self.expression == other.expression).all()
            else:
                return self.identifiers == other.identifiers and \
                       self.conditions == other.conditions and \
                       (self._p_values == other._p_values).all() and \
                       (self.expression == other.expression).all()

    @classmethod
    def from_data_frame(cls, data_frame):
        """
        Read expression data from a pandas.DataFrame.
        Parameters
        ----------
        data_frame: pandas.DataFrame
            A DataFrame containing the genes as index and
            the columns as conditions.
            For more information about p-values see
            @ExpressionProfile.p_value_columns
        Returns
        -------
        ExpressionProfile
        """
        columns = data_frame.columns.tolist()
        conditions = [c for c in columns if "p-value" not in c]
        p_value_keys = [c for c in columns if "p-value" in c]
        if p_value_keys:
            p_values = data_frame[p_value_keys].values
        else:
            p_values = None

        expression = data_frame[conditions].values
        identifiers = data_frame.index.tolist()
        return ExpressionProfile(identifiers, conditions, expression, p_values)


    @property
    def data_frame(self):
        """
        Build a pandas.DataFrame from the ExpressionProfile.
        Returns
        -------
        pandas.DataFrame
        """
        if self._p_values is None:
            expression = self.expression
            conditions = self.conditions
        else:
            expression = np.concatenate((self.expression, self.p_values),
                                        axis=1)
            conditions = self.conditions + self.p_value_columns

        return pd.DataFrame(expression,
                            index=self.identifiers,
                            columns=conditions)

    @property
    def p_value_columns(self):
        """
        Generate the p-value column names.
        Returns
        -------
        list
        """
        return ["{} {} p-value".format(c[0], c[1])
                for c in combinations(self.conditions, 2)]

    @property
    def p_values(self):
        """
        Return the p-values between the conditions.
        Returns
        -------
        numpy.ndarray
        Raises
        ------
        ValueError
        """
        if not self._p_values.all():
            raise ValueError("No p-values defined.")
        else:
            return self._p_values

    @p_values.setter
    def p_values(self, p_values):
        """
        Set p_values.
        Parameters
        ----------
        p_values: numpy.ndarray
        """
        assert isinstance(p_values, (np.ndarray, type(None)))
        if p_values is not None:
            if p_values.shape[1] != len(self.p_value_columns):
                raise ValueError("Argument p-values do not cover all \
                                  conditions")

        self._p_values = p_values

    @p_values.deleter
    def p_values(self):
        """Delete p_values."""
        self._p_values = None

    def differences(self, p_value=0.005):
        """
        Calculate the differences based on the MADE method.
        Parameters
        ----------
        p_value: float, optional (default 0.005)
            A p-value to set as cutoff for calculation.
        Returns
        -------
        dict
        """
        diff = {}
        for idx, iden in enumerate(self.identifiers):
            diff[iden] = []
            for i in range(1, len(self.conditions)):
                start, end = self.expression[idx, i-1: i+1]
                p_val = self.p_values[idx, i-1]
                if p_val <= p_value:
                    if start < end:
                        diff[iden].append(+1)
                    elif start > end:
                        diff[iden].append(-1)
                    else:
                        diff[iden].append(0)
                else:
                    diff[iden].append(0)
        return diff

    def _map_gene_to_rxn(self, reaction, gene_values, by):
        """
        Map gene data to reactions (using GPR associations).
        Parameters
        ----------
        reaction: cobra.Reaction
            The reaction to map gene data to.
        gene_values: dict
            The dict of gene IDs as keys and expression values as values.
        by: str
            The method to use for mapping the gene data.
            "or2max_and2min": if more than one gene contribute as a complex,
                              the min of the gene values is taken and if the
                              genes act as isozymes, the max is taken.
            "or2sum_and2min": if more than one gene contribute as a complex,
                              the min of the gene values is taken and if the
                              genes act as isozymes, the sum is taken.
        Returns
        -------
        float
        """
        local_dict = {gene.id: Symbol(gene.id) for gene in reaction.genes}
        rule = reaction.gene_reaction_rule.replace("and", "*").replace("or",
                                                                       "+")
        expression = parse_expr(rule, local_dict)
        if by == "or2max_and2min":
            expression = expression.replace(Mul, Min).replace(Add, Max)
        elif by == "or2sum_and2min":
            expression = expression.replace(Mul, Min)
        return expression.subs(gene_values).evalf()

    def to_dict(self, condition):
        """
        Build a dict with identifiers and the expression for the condition.
        Parameters
        ----------
        condition: str or int
            The condition label or the column index.
        Returns
        -------
        dict
        """
        if isinstance(condition, int):
            index = condition
        else:
            index = self._condition_index[condition]
        return {identifier: expression for identifier, expression in
                zip(self.identifiers, self.expression[:, index])}

    def to_reaction_dict(self, condition, model, **kwargs):
        """
        Build a dict with reactions and the expression for the condition.
        If identifiers in the profile are genes, GPR associations are
        respected. If the gene expression data is missing in the profile,
        the reaction(s) related to that gene, is(are) supplied with the cutoff
        value.
        Parameters
        ----------
        condition: str or int
            The condition or the column index.
        model: cobra.Model
            The constraint-based model to obtain the GPR associations from.
        map_by: str, optional (default "or2max_and2min")
            The method to use for mapping gene values to reactions.
        cutoff: float, optional (default 0.0)
            The value to set for reaction(s) whose gene lacks a value in
            the profile.
        Returns
        -------
        dict
        """
        cutoff = 0.0
        map_by = "or2max_and2min"
        gene_exp = self.to_dict(condition)
        rxn_exp = {}
        for rxn in model.reactions:
            if rxn.genes is not None and any([gene.id in self.identifiers
                                              for gene in rxn.genes]):
                gene_values = {gene.id: gene_exp.get(gene.id, cutoff)
                               for gene in rxn.genes}
                rxn_exp[rxn.id] = self._map_gene_to_rxn(rxn, gene_values,
                                                        by=map_by)
        return rxn_exp

    def minmax(self, condition=None):
        """
        Return the min and max values for the specified condition.
        Parameters
        ----------
        condition: str or int or None, optional (default None)
            The condition to obtain the min and max values for.
        Returns
        -------
        tuple of (min, max)
        """
        if condition is None:
            values = self[:, :]
        elif isinstance(condition, int):
            values = self[:, condition]
        elif isinstance(condition, str):
            values = self[:, self._condition_index[condition]]
        return np.amin(values), np.amax(values)

    def _repr_html_(self):
        return self.data_frame._repr_html_()
    
from __future__ import absolute_import

from optlang.symbolics import Zero
from six import iteritems

In [65]:
exp = np.array([[.1,1,2],[.1,1,2],[.1,1,2],[.1,1,2],[.1,1,2],[.5,1,.5]])
pval = np.array([[0.05,0.05],[0.05,0.05],[0.05,0.05],[0.05,0.05],[0.05,0.05],[0.05,0.05],[.5,.5]])

new_obj = ExpressionProfile(identifiers = ['b1702','b4151','b4152','b4153','b4154','b2284'],
                               conditions = ['control','experimental_1','experimental_2'],
                               expression = exp)

new_obj

,control,experimental_1,experimental_2
b1702,0.1,1.0,2.0
b4151,0.1,1.0,2.0
b4152,0.1,1.0,2.0
b4153,0.1,1.0,2.0
b4154,0.1,1.0,2.0
b2284,0.5,1.0,0.5


In [69]:
new_obj._condition_index

{'control': 0, 'experimental_1': 1, 'experimental_2': 2}

In [70]:
new_obj.identifier_index

{'b1702': 0, 'b2284': 5, 'b4151': 1, 'b4152': 2, 'b4153': 3, 'b4154': 4}

In [68]:

solution = model.slim_optimize()
prob = model.problem
rxn_profile = new_obj.to_reaction_dict(condition, model)

rxn_profile

# if model.objective_direction == 'max':
#     fix_obj_const = prob.Constraint(model.objective.expression,
#                                     lb=fraction_of_optimum * solution,
#                                     name="RMF")
# else:
#     fix_obj_const = prob.Constraint(model.objective.expression,
#                                     ub=fraction_of_optimum * solution,
#                                     name="RMF")
# model.add_cons_vars(fix_obj_const)

KeyError: 'San_Diego'

In [23]:
# inspired by NEW GIMME IN DRIVEN


prob = model.problem
rxn_profile = expression_profile.to_reaction_dict(condition, model)

solution = model.slim_optimize()
fix_obj_const = prob.Constraint(model.objective.expression,
                                lb=fraction_of_optimum * solution, name="RMF")

model.add_cons_vars(fix_obj_const)

coefficients = {rxn_id: cutoff - expression
                for rxn_id, expression in iteritems(rxn_profile)
                if cutoff > expression}
obj_vars = []
for rxn_id, coefficient in iteritems(coefficients):
    rxn = model.reactions.get_by_id(rxn_id)
    obj_vars.append((rxn.forward_variable, coefficient))
    obj_vars.append((rxn.reverse_variable, coefficient))

model.objective = prob.Objective(Zero, sloppy=True, direction="min")
model.objective.set_linear_coefficients({v: c for v, c in obj_vars})
sol = model.optimize()


{0.0 <= DM_4CRSOL <= 1000.0: 1.0,
 0 <= DM_4CRSOL_reverse_50477 <= 0: 1.0,
 0.0 <= DM_5DRIB <= 1000.0: 1.0,
 0 <= DM_5DRIB_reverse_0b428 <= 0: 1.0,
 0.0 <= DM_AACALD <= 1000.0: 1.0,
 0 <= DM_AACALD_reverse_73112 <= 0: 1.0,
 0.0 <= DM_AMOB <= 1000.0: 1.0,
 0 <= DM_AMOB_reverse_9e172 <= 0: 1.0,
 0.0 <= DM_MTHTHF <= 1000.0: 1.0,
 0 <= DM_MTHTHF_reverse_78338 <= 0: 1.0,
 0.0 <= DM_OXAM <= 1000.0: 1.0,
 0 <= DM_OXAM_reverse_7c3e7 <= 0: 1.0,
 0.0 <= Ec_biomass_iJO1366_WT_53p95M <= 1000.0: 1.0,
 0 <= Ec_biomass_iJO1366_WT_53p95M_reverse_55db7 <= 0: 1.0,
 0.0 <= Ec_biomass_iJO1366_core_53p95M <= 1000.0: 1.0,
 0 <= Ec_biomass_iJO1366_core_53p95M_reverse_e94eb <= 0: 1.0,
 0.0 <= EX_12ppd__R_e <= 1000.0: 1.0,
 0 <= EX_12ppd__R_e_reverse_cb334 <= 0: 1.0,
 0.0 <= EX_12ppd__S_e <= 1000.0: 1.0,
 0 <= EX_12ppd__S_e_reverse_6f659 <= 0: 1.0,
 0.0 <= EX_14glucan_e <= 1000.0: 1.0,
 0 <= EX_14glucan_e_reverse_4b919 <= 0: 1.0,
 0.0 <= EX_15dap_e <= 1000.0: 1.0,
 0 <= EX_15dap_e_reverse_020d4 <= 0: 1.0,
 0.0

In [6]:
from optlang import Variable, Constraint, Objective, Model

variables = {}
constraints = {}
for condition in conditions: # 0 = Control, 1 = Experimental
    variables[condition] = {}
    constraints[condition] = {}
    for gene in model.genes:
        var = Variable(name="{}_state".format(gene.id), lb=0, type="integer")
        variables[condition][gene.id] = var

        if gene.id in increased:
            const = Constraint((-log(pval[gene.id]))*(variables[conditions[0]][gene.id] - variables[conditions[1]][gene.id])
                               lb = 0,
                               name = '{}_'.format(gene.id))
        if gene.id in decreased:
            const = Constraint((-log(pval[gene.id]))*(variables[conditions[1]][gene.id] - variables[conditions[0]][gene.id])
                               lb = 0,
                               name = '{}_'.format(gene.id))
        constraints[condition][gene.id] = const

NameError: name 'log' is not defined

In [ ]:

# We can print the objective and constraints
print(obj)
print("")
for const in constraints:
    print(const)

print("")

# Put everything together in a Model
model = Model()
model.add(constraints)  # Variables are added implicitly
model.objective = obj

# Optimize and print the solution
status = model.optimize()
print("Status:", status)
print("Objective value:", model.objective.value)
print("")
for var in model.variables:
    print(var.name, ":", var.primal)

In [5]:
from optlang import Model, Variable, Constraint, Objective

# All the (symbolic) variables are declared, with a name and optionally a lower and/or upper bound.
for gene in model.genes:
    x = Variable('gene_state'+gene.id, lb = 0)
    c1 = Constraint(abs(x-exp_dif), ub = 1)
    c2 = Constraint((1-log(pval))*(abs(x-exp_dif)))
    
    
x1 = Variable('gene_state_1', lb=0)
x2 = Variable('gene_state_2', lb=0)
x3 = Variable('gene_state_3', lb=0)



# A constraint is constructed from an expression of variables and a lower and/or upper bound (lb and ub).
c1 = Constraint(x1 + x2 + x3, ub=100)
c2 = Constraint(10 * x1 + 4 * x2 + 5 * x3, ub=600)
c3 = Constraint(2 * x1 + 2 * x2 + 6 * x3, ub=300)

# An objective can be formulated
obj = Objective(10 * x1 + 6 * x2 + 4 * x3, direction='max')

# Variables, constraints and objective are combined in a Model object, which can subsequently be optimized.
model = Model(name='Simple model')
model.objective = obj
model.add([c1, c2, c3])
status = model.optimize()

In [2]:
# add constraint framework
add_pfba(model)

# examine constraints
coef = model.objective.get_linear_coefficients(model.variables)
coef

{0.0 <= DM_4CRSOL <= 1000.0: 1.0,
 0 <= DM_4CRSOL_reverse_50477 <= 0: 1.0,
 0.0 <= DM_5DRIB <= 1000.0: 1.0,
 0 <= DM_5DRIB_reverse_0b428 <= 0: 1.0,
 0.0 <= DM_AACALD <= 1000.0: 1.0,
 0 <= DM_AACALD_reverse_73112 <= 0: 1.0,
 0.0 <= DM_AMOB <= 1000.0: 1.0,
 0 <= DM_AMOB_reverse_9e172 <= 0: 1.0,
 0.0 <= DM_MTHTHF <= 1000.0: 1.0,
 0 <= DM_MTHTHF_reverse_78338 <= 0: 1.0,
 0.0 <= DM_OXAM <= 1000.0: 1.0,
 0 <= DM_OXAM_reverse_7c3e7 <= 0: 1.0,
 0.0 <= Ec_biomass_iJO1366_WT_53p95M <= 1000.0: 1.0,
 0 <= Ec_biomass_iJO1366_WT_53p95M_reverse_55db7 <= 0: 1.0,
 0.0 <= Ec_biomass_iJO1366_core_53p95M <= 1000.0: 1.0,
 0 <= Ec_biomass_iJO1366_core_53p95M_reverse_e94eb <= 0: 1.0,
 0.0 <= EX_12ppd__R_e <= 1000.0: 1.0,
 0 <= EX_12ppd__R_e_reverse_cb334 <= 0: 1.0,
 0.0 <= EX_12ppd__S_e <= 1000.0: 1.0,
 0 <= EX_12ppd__S_e_reverse_6f659 <= 0: 1.0,
 0.0 <= EX_14glucan_e <= 1000.0: 1.0,
 0 <= EX_14glucan_e_reverse_4b919 <= 0: 1.0,
 0.0 <= EX_15dap_e <= 1000.0: 1.0,
 0 <= EX_15dap_e_reverse_020d4 <= 0: 1.0,
 0.0

In [4]:
for rxn in model.reactions:
    if rxn is expressed:
        expression_constraints[rxn] = {'expression': expression_change,
                                      'p_val' : expression_pval}
    else:
        expression_constraints[rxn] = {'expression': 1,
                                      'p_val' : 1}
        
for key,value in coef.items():
    print(key.name)
    print(value)
    minimize this: (1-log(expression_constraints[key.name][p_val]))*(abs(state- expression_constraints[key.name][expression]))
#     if key.name.startswith('DM_') or key.name.startswith('SK_'):
#             coef[key] = 1000.

DM_4CRSOL
1.0
DM_4CRSOL_reverse_50477
1.0
DM_5DRIB
1.0
DM_5DRIB_reverse_0b428
1.0
DM_AACALD
1.0
DM_AACALD_reverse_73112
1.0
DM_AMOB
1.0
DM_AMOB_reverse_9e172
1.0
DM_MTHTHF
1.0
DM_MTHTHF_reverse_78338
1.0
DM_OXAM
1.0
DM_OXAM_reverse_7c3e7
1.0
Ec_biomass_iJO1366_WT_53p95M
1.0
Ec_biomass_iJO1366_WT_53p95M_reverse_55db7
1.0
Ec_biomass_iJO1366_core_53p95M
1.0
Ec_biomass_iJO1366_core_53p95M_reverse_e94eb
1.0
EX_12ppd__R_e
1.0
EX_12ppd__R_e_reverse_cb334
1.0
EX_12ppd__S_e
1.0
EX_12ppd__S_e_reverse_6f659
1.0
EX_14glucan_e
1.0
EX_14glucan_e_reverse_4b919
1.0
EX_15dap_e
1.0
EX_15dap_e_reverse_020d4
1.0
EX_23camp_e
1.0
EX_23camp_e_reverse_60590
1.0
EX_23ccmp_e
1.0
EX_23ccmp_e_reverse_3700b
1.0
EX_23cgmp_e
1.0
EX_23cgmp_e_reverse_48b5f
1.0
EX_23cump_e
1.0
EX_23cump_e_reverse_f767c
1.0
EX_23dappa_e
1.0
EX_23dappa_e_reverse_dfe54
1.0
EX_26dap__M_e
1.0
EX_26dap__M_e_reverse_a7bf0
1.0
EX_2ddglcn_e
1.0
EX_2ddglcn_e_reverse_e7cfb
1.0
EX_34dhpac_e
1.0
EX_34dhpac_e_reverse_01c3a
1.0
EX_3amp_e
1.0
EX_3amp_

1.0
ACACT1r
1.0
ACACT1r_reverse_7e2ab
1.0
ACACT2r
1.0
ACACT2r_reverse_b794d
1.0
ACACT3r
1.0
ACACT3r_reverse_b8079
1.0
ACACT4r
1.0
ACACT4r_reverse_36b94
1.0
ACACT5r
1.0
ACACT5r_reverse_49fec
1.0
ACACT6r
1.0
ACACT6r_reverse_a3ce9
1.0
ACACT7r
1.0
ACACT7r_reverse_b44b4
1.0
ACACT8r
1.0
ACACT8r_reverse_54705
1.0
ACACt2pp
1.0
ACACt2pp_reverse_06302
1.0
ACACtex
1.0
ACACtex_reverse_cc949
1.0
ACALD
1.0
ACALD_reverse_fda2b
1.0
ACALDtex
1.0
ACALDtex_reverse_75f7f
1.0
ACALDtpp
1.0
ACALDtpp_reverse_52ae6
1.0
ACANTHAT
1.0
ACANTHAT_reverse_9fe7b
1.0
ACBIPGT
1.0
ACBIPGT_reverse_bcc45
1.0
ACCOAC
1.0
ACCOAC_reverse_9d1cd
1.0
ACCOAL
1.0
ACCOAL_reverse_ea444
1.0
ACGAL1PPpp
1.0
ACGAL1PPpp_reverse_88026
1.0
ACGAL1Ptex
1.0
ACGAL1Ptex_reverse_c2cd1
1.0
ACGALtex
1.0
ACGALtex_reverse_54263
1.0
ACGAM1PPpp
1.0
ACGAM1PPpp_reverse_fbc31
1.0
ACGAM1Ptex
1.0
ACGAM1Ptex_reverse_d95c0
1.0
ACGAMK
1.0
ACGAMK_reverse_1e7a8
1.0
ACGAMT
1.0
ACGAMT_reverse_2307a
1.0
ACGAptspp
1.0
ACGAptspp_reverse_e1a6e
1.0
ACGAtex
1.0
ACGAtex_

1.0
DRPA_reverse_66bfb
1.0
DSBAO1
1.0
DSBAO1_reverse_9c7d6
1.0
DSBAO2
1.0
DSBAO2_reverse_f2c04
1.0
DSBCGT
1.0
DSBCGT_reverse_58858
1.0
DSBDR
1.0
DSBDR_reverse_7e26e
1.0
DSBGGT
1.0
DSBGGT_reverse_f8e4c
1.0
DSERDHr
1.0
DSERDHr_reverse_c44ee
1.0
DSERt2pp
1.0
DSERt2pp_reverse_d3685
1.0
DSERtex
1.0
DSERtex_reverse_5ab31
1.0
DTARTD
1.0
DTARTD_reverse_0d68b
1.0
DTMPK
1.0
DTMPK_reverse_44d5a
1.0
DTMPtex
1.0
DTMPtex_reverse_581e0
1.0
DUMPtex
1.0
DUMPtex_reverse_a35ee
1.0
DURADx
1.0
DURADx_reverse_224c5
1.0
DURIK1
1.0
DURIK1_reverse_f4aee
1.0
DURIPP
1.0
DURIPP_reverse_e8f8a
1.0
DURIt2pp
1.0
DURIt2pp_reverse_a9066
1.0
DURItex
1.0
DURItex_reverse_bdbc1
1.0
DUTPDP
1.0
DUTPDP_reverse_1eccd
1.0
DXPRIi
1.0
DXPRIi_reverse_85956
1.0
DXPS
1.0
DXPS_reverse_86aca
1.0
DXYLK
1.0
DXYLK_reverse_fb109
1.0
E4PD
1.0
E4PD_reverse_babdb
1.0
EAR100x
1.0
EAR100x_reverse_d973f
1.0
EAR100y
1.0
EAR100y_reverse_863b6
1.0
EAR120x
1.0
EAR120x_reverse_72a18
1.0
EAR120y
1.0
EAR120y_reverse_c7353
1.0
EAR121x
1.0
EAR121x_rever

1.0
LPLIPAL1G141pp_reverse_30b2b
1.0
LPLIPAL1G160pp
1.0
LPLIPAL1G160pp_reverse_6ab0f
1.0
LPLIPAL1G161pp
1.0
LPLIPAL1G161pp_reverse_aecac
1.0
LPLIPAL1G180pp
1.0
LPLIPAL1G180pp_reverse_9b51e
1.0
LPLIPAL1G181pp
1.0
LPLIPAL1G181pp_reverse_c46f5
1.0
LPLIPAL2A120
1.0
LPLIPAL2A120_reverse_844c0
1.0
LPLIPAL2A140
1.0
LPLIPAL2A140_reverse_6e1ff
1.0
LPLIPAL2A141
1.0
LPLIPAL2A141_reverse_12ad1
1.0
LPLIPAL2A160
1.0
LPLIPAL2A160_reverse_b2af0
1.0
LPLIPAL2A161
1.0
LPLIPAL2A161_reverse_37df8
1.0
LPLIPAL2A180
1.0
LPLIPAL2A180_reverse_dba15
1.0
LPLIPAL2A181
1.0
LPLIPAL2A181_reverse_8b966
1.0
LPLIPAL2ATE120
1.0
LPLIPAL2ATE120_reverse_deb80
1.0
LPLIPAL2ATE140
1.0
LPLIPAL2ATE140_reverse_0c69d
1.0
LPLIPAL2ATE141
1.0
LPLIPAL2ATE141_reverse_eae4b
1.0
LPLIPAL2ATE160
1.0
LPLIPAL2ATE160_reverse_bcf82
1.0
LPLIPAL2ATE161
1.0
LPLIPAL2ATE161_reverse_a516f
1.0
LPLIPAL2ATE180
1.0
LPLIPAL2ATE180_reverse_cce82
1.0
LPLIPAL2ATE181
1.0
LPLIPAL2ATE181_reverse_e8332
1.0
LPLIPAL2ATG120
1.0
LPLIPAL2ATG120_reverse_9d5c1
1.0
LPL

1.0
S4FE4ST
1.0
S4FE4ST_reverse_caa0d
1.0
S7PI
1.0
S7PI_reverse_6ace9
1.0
SADH
1.0
SADH_reverse_a0164
1.0
SADT2
1.0
SADT2_reverse_2632d
1.0
SARCOX
1.0
SARCOX_reverse_eebad
1.0
SBTPD
1.0
SBTPD_reverse_9a7da
1.0
SBTptspp
1.0
SBTptspp_reverse_05c76
1.0
SBTtex
1.0
SBTtex_reverse_6aeda
1.0
SCYSDS
1.0
SCYSDS_reverse_3bb75
1.0
SDPDS
1.0
SDPDS_reverse_43d25
1.0
SDPTA
1.0
SDPTA_reverse_76834
1.0
SELCYSS
1.0
SELCYSS_reverse_2f401
1.0
SELGTHR
1.0
SELGTHR_reverse_87696
1.0
SELGTHR2
1.0
SELGTHR2_reverse_076d8
1.0
SELGTHR3
1.0
SELGTHR3_reverse_02c0a
1.0
SELNPS
1.0
SELNPS_reverse_498bb
1.0
SELR
1.0
SELR_reverse_c1bf1
1.0
SELtex
1.0
SELtex_reverse_316ff
1.0
SELtpp
1.0
SELtpp_reverse_466c6
1.0
SEPHCHCS
1.0
SEPHCHCS_reverse_cb185
1.0
SERASr
1.0
SERASr_reverse_eb9c5
1.0
SERAT
1.0
SERAT_reverse_0de5e
1.0
SERD_D
1.0
SERD_D_reverse_0aea1
1.0
SERD_L
1.0
SERD_L_reverse_0f0ab
1.0
SERTRS
1.0
SERTRS_reverse_b65b1
1.0
SERTRS2
1.0
SERTRS2_reverse_0c19b
1.0
SERt2rpp
1.0
SERt2rpp_reverse_94979
1.0
SERt4pp
1.0
SERt4p

In [ ]:


for x in model.variables: 
    if x.name.startswith('DM_') or x.name.startswith('SK_') or x.name.startswith('EX_'):
        model.objective.set_linear_coefficients(coef)

# Set your desired bound on the original objective (here, biomass, which has been added to 
# the universal model from the model to be gapfilled)
model.reactions.get_by_id(objective_reaction).lower_bound = 0.80

# optimize and grab active reactions. 1E-8 is an arbitrary threshold, and in my example nothing
# had fluxes between 1E-5 and 1E-10, suggesting the threshold isn't as big of a deal as in the
# MILP formulation. This code will also return flux through exchange reactions (and any reaction
# from the model that was not present in the universal; you'll have to find another way to filter these).
solution = model.optimize()
get_fluxes = set([r.id for r in model.reactions])
add_reactions_to_model = [rxn for rxn in get_fluxes if abs(solution.x_dict[rxn]) > 1E-8]

